# PlayHQ Season Team & Fixture Extraction

This notebook will help extract all teams and fixture so far from from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs).

Used for the scheduler season capstone project.

Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [5]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import json
import re
import os
import csv

import playhq as phq

## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [9]:
# Change this to import your club's own configuration
from config_bmc_s22 import *
# from config_cba import *

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)

print("Timezone:", TIMEZONE)
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)
OUTPUT_PATH = "output_02"

Club name: Brunswick Magic Basketball Club (org. id: 8c4d5431-eaa5-4644-82ac-992abe224b88)
X-tenant: bv x-api-key: f5d33c76-f858-49fa-8330-8e0e396219cd
Timezone: Australia/Melbourne
PlayHQ Club fixture: https://bit.ly/bmbc-s22


## Get seasons

In [10]:
# get competition id
SEASON = "Winter 2023"
# SEASON = None   # use this if you don't know and want to list them all

season_id = None
competition_id = None
for data_json in phq_club.get_json(f"organisations/{ORG_ID}/seasons"):
    # print(json.dumps(data_json, sort_keys=True, indent=4))

    for x in data_json['data']:
        if SEASON is None:
            continue
        if x['name'] == SEASON:
            season_id = x['id']
            # competition_id = x['competition']['id']
            print(f'Seasons *{SEASON}* found with id: {season_id}')



Seasons *Winter 2023* found with id: cdbe3065-2a32-4c6d-8771-f8fae3fa7611


## Get teams

In [13]:
season_id = "a94981b4-75b7-429f-9005-915182ab6153" # Victorian Junior Domestic - Summer 2022/23
season_id = "cdbe3065-2a32-4c6d-8771-f8fae3fa7611" # Victorian Junior Domestic - Winter 2023
# season_id = "78824ad1-0ca6-46c5-9440-dbe20c948b2f"  # VJBL 2023
# season_id = "8d724c0a-3a0f-4da7-92f8-2e2053a6707b"  # National Junior 2023

teams = []
for data_json in phq_club.get_json(f"seasons/{season_id}/teams"):
    if data_json['data'][0]['club'] is None:
        break
    # print(data_json)
    for t in data_json['data']:
        # print(t)
        team = {}

        team['id'] = t['id']
        team['club'] = t['club']['name'] if t['club'] is not None else "Unknown"
        if t['grade'] is None:
            continue    # skip teams without grade
        team['grade'] = t['grade']['name']
        team['name'] = t['name']
        
        teams.append(team)
    # print(json.dumps(data_json, sort_keys=True, indent=4))

print("Number of teams extracted:", len(teams))
print(teams[10])


fieldnames = ["id", "club", "grade", "name"]
file_name = os.path.join(OUTPUT_PATH, "teams.csv") 
with open(file_name, 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(teams)


Number of teams extracted: 306
{'id': '139ecf7c-55e5-411f-bbce-4bceb321f342', 'club': 'Northcote Jets Basketball Club Inc.', 'grade': 'Juniors U14 Boys Div 3', 'name': 'Jets U14 Boys Yellow'}


In [12]:
# team_id = "139ecf7c-55e5-411f-bbce-4bceb321f342"
# team_id="b55cb46c-8415-4b6e-9ac5-ad38b02e5b35"

for team in teams:
    team_id = team['id']
    standard_name = re.sub(r'[\W_]', "_", team['name'])
    file_name = os.path.join(OUTPUT_PATH, standard_name + ".json") 

    if os.path.exists(file_name):
        continue
    print("Processing team:", team['name'])

    games = {}
    games["data"] = []

    for data_json in phq_club.get_json(f"teams/{team_id}/fixture"):
        # if data_json['data'][0]['club'] is None:
        #     break
        games["data"] = games["data"] + data_json['data']

    with open(file_name, "w") as f:
        f.write(json.dumps(games, sort_keys=True, indent=4))


print("Number of teams fixture extracted:", len(teams))
# print(teams[10])

Processing team: Jets U10 Boys Blue
Processing team: Jets U10 Boys Yellow
Processing team: Jets U12 Boys Blue
Processing team: Jets U12 Boys Yellow
Processing team: Jets U12 Girls Blue
Processing team: Jets U12 Girls Yellow
Processing team: Jets U14 Boys Blue
Processing team: Jets U14 Boys Green
Processing team: Jets U14 Boys Orange
Processing team: Jets U14 Boys Red
Processing team: Jets U14 Boys Yellow
Processing team: Jets U14 Girls Blue
Processing team: Jets U14 Girls Yellow
Processing team: Jets U16 Boys Blue
Processing team: Jets U16 Boys Orange
Processing team: Jets U16 Boys Yellow
Processing team: Jets U16 Girls Blue
Processing team: Jets U16 Girls Yellow
Processing team: Jets U18 Boys Blue
Processing team: Jets U18 Boys Orange
Processing team: Jets U18 Boys Yellow
Processing team: Jets U18 Girls Yellow
Processing team: Jets U20 Boys Blue
Processing team: Jets U20 Girls Blue
Processing team: Magic U10 Boys Black
Processing team: Magic U10 Boys Gold
Processing team: Magic U10 Bo